### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import OfficeHomeDataset
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

oh = OfficeHomeDataset(
    data_root=CFG["datasets"]["OfficeHome"]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = "OfficeHome" 
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-10 17:51:34,890 | INFO | Initialized experiment directories for OfficeHome
2025-10-10 17:51:34,892 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\OfficeHome\logs
2025-10-10 17:51:34,892 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\OfficeHome\checkpoints
2025-10-10 17:51:34,892 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\OfficeHome\plots


### Setup

In [4]:
domains = CFG["datasets"]["OfficeHome"]["domains"]
loaders = {d: {"train": oh.get_dataloader(d, train=True), "val": oh.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset="OfficeHome")
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key="OfficeHome",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-10 17:51:35,271 | INFO | === LODO: Leaving out domain 'Art' ===



=== LODO: Leaving out domain 'Art' ===


Evaluating: 100%|██████████| 19/19 [00:14<00:00,  1.36it/s]
2025-10-10 17:52:23,127 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.4442, Cls: 2.4358, GRQO: 0.0084, Acc: 0.4679 | Val - Loss: 1.9780, Cls: 1.9755, GRQO: 0.0025, Acc: 0.5262
2025-10-10 17:52:23,304 | INFO | [Art] New best val acc: 0.5262


[Art] Epoch 1/5 | Train - Loss: 2.4442, Cls: 2.4358, GRQO: 0.0084, Acc: 0.4679 | Val - Loss: 1.9780, Cls: 1.9755, GRQO: 0.0025, Acc: 0.5262
[Art] New best val acc: 0.5262


Evaluating: 100%|██████████| 19/19 [00:13<00:00,  1.46it/s]
2025-10-10 17:53:11,667 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.7455, Cls: 0.7413, GRQO: 0.0042, Acc: 0.8268 | Val - Loss: 1.8144, Cls: 1.8128, GRQO: 0.0015, Acc: 0.5501
2025-10-10 17:53:11,778 | INFO | [Art] New best val acc: 0.5501


...

[Real World] Epoch 3/5 | Train - Loss: 0.3619, Cls: 0.3583, GRQO: 0.0036, Acc: 0.9231 | Val - Loss: 1.0929, Cls: 1.0921, GRQO: 0.0008, Acc: 0.7122
[Real World] New best val acc: 0.7122


Evaluating: 100%|██████████| 35/35 [00:21<00:00,  1.62it/s]
2025-10-10 18:04:06,554 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.1503, Cls: 0.1472, GRQO: 0.0031, Acc: 0.9751 | Val - Loss: 1.0561, Cls: 1.0555, GRQO: 0.0007, Acc: 0.7285
2025-10-10 18:04:06,623 | INFO | [Real World] New best val acc: 0.7285


[Real World] Epoch 4/5 | Train - Loss: 0.1503, Cls: 0.1472, GRQO: 0.0031, Acc: 0.9751 | Val - Loss: 1.0561, Cls: 1.0555, GRQO: 0.0007, Acc: 0.7285
[Real World] New best val acc: 0.7285


Evaluating: 100%|██████████| 35/35 [00:21<00:00,  1.63it/s]
2025-10-10 18:04:46,102 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.0775, Cls: 0.0748, GRQO: 0.0027, Acc: 0.9860 | Val - Loss: 1.0607, Cls: 1.0602, GRQO: 0.0005, Acc: 0.7315
2025-10-10 18:04:46,174 | INFO | [Real World] New best val acc: 0.7315
2025-10-10 18:04:46,174 | INFO | [Real World] Best Acc: 0.7315
2025-10-10 18:04:46,175 | INFO | ------------------------------------------------------------
2025-10-10 18:04:46,176 | INFO | LODO finished | Mean Acc: 0.6265 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\OfficeHome\logs\lodo_summary_20251010_180446.json


[Real World] Epoch 5/5 | Train - Loss: 0.0775, Cls: 0.0748, GRQO: 0.0027, Acc: 0.9860 | Val - Loss: 1.0607, Cls: 1.0602, GRQO: 0.0005, Acc: 0.7315
[Real World] New best val acc: 0.7315
[Real World] Best Acc: 0.7315
------------------------------------------------------------
LODO finished | Mean Acc: 0.6265
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\OfficeHome\logs\lodo_summary_20251010_180446.json


### Baseline

In [6]:
model_name ="resnet18"
baseline_results, baseline_mean = run_baseline(
    model_name=model_name,
    CFG=CFG,
    logger=logger,
    dataset_key="OfficeHome",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-10 18:04:46,188 | INFO | Initializing ResNet baseline: resnet18
2025-10-10 18:04:46,277 | INFO | === Baseline LODO: Leaving out domain 'Art' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'Art' ===


2025-10-10 18:05:22,253 | INFO | [Art] Epoch 1/5 | Train - Loss: 2.2092, Acc: 0.5282 | Val Acc: 0.4878


[Art] Epoch 1/5 | Train - Loss: 2.2092, Acc: 0.5282 | Val Acc: 0.4878


2025-10-10 18:05:59,138 | INFO | [Art] Epoch 2/5 | Train - Loss: 0.8353, Acc: 0.8375 | Val Acc: 0.5373


...

[Real World] Epoch 3/5 | Train - Loss: 0.5272, Acc: 0.9096 | Val Acc: 0.7487


2025-10-10 18:17:47,951 | INFO | [Real World] Epoch 4/5 | Train - Loss: 0.2607, Acc: 0.9660 | Val Acc: 0.7505


[Real World] Epoch 4/5 | Train - Loss: 0.2607, Acc: 0.9660 | Val Acc: 0.7505


2025-10-10 18:18:28,610 | INFO | [Real World] Epoch 5/5 | Train - Loss: 0.1321, Acc: 0.9842 | Val Acc: 0.7519
2025-10-10 18:18:28,616 | INFO | [Real World] Best Val Acc: 0.7519
2025-10-10 18:18:28,616 | INFO | ------------------------------------------------------------
2025-10-10 18:18:28,618 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.6290


[Real World] Epoch 5/5 | Train - Loss: 0.1321, Acc: 0.9842 | Val Acc: 0.7519
[Real World] Best Val Acc: 0.7519
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.6290
